In [1]:
import requests
import zipfile
import re
import random

In [2]:
r = requests.get("https://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip")
with open("sms.zip", "wb") as f:
    f.write(r.content)

zipfile.ZipFile("sms.zip").extractall() 

data = []
with open("SMSSpamCollection.txt", "r") as f:
    for line in f:
        data.append(line.strip().split('\t'))

for d in data:
    d[1] = set(re.findall("[0-9a-z_]+", d[1].lower()))

In [3]:
random.shuffle(data)

train_size = int(len(data) * 0.8)
test_zise = len(data) - train_size

train = data[:train_size]
test = data[train_size:]

In [4]:
n_total = len(train)
n_spam = sum(1 for cls, bow in train if cls == "spam") 
n_ham = sum(1 for cls, bow in train if cls == "ham")

alpha = 0.5

prior_spam = (n_spam + alpha) / (n_total + 2*alpha)
prior_ham = (n_ham + alpha) / (n_total + 2*alpha)

print(prior_ham, prior_spam)

0.8702914798206278 0.1297085201793722


In [5]:
spam_words = {}
ham_words = {}

for cls, bow in train:
    for word in bow:
        if cls == "spam":
            if word not in spam_words:
                spam_words[word] = 0
            spam_words[word] += 1
        if cls == "ham":
            if word not in ham_words:
                ham_words[word] = 0
            ham_words[word] += 1

In [6]:
from collections import Counter
import math

spam_words = Counter(word for cls, bow in train 
                    for word in bow 
                    if cls=="spam")
ham_words = Counter(word for cls, bow in train 
                    for word in bow 
                    if cls=="ham")

In [7]:
def predict(bow, spam_words, ham_words, n_spams, n_hams, prior_spam, prior_ham, alpha):
    spam_score = math.log(prior_spam)
    for word in bow:
        spam_score += math.log((spam_words[word] + alpha) / (n_spams + alpha))
    
    ham_score = math.log(prior_ham)
    for word in bow:
        ham_score += math.log((ham_words[word] + alpha) / (n_hams + alpha))
    
    if spam_score < ham_score:
        return 'ham'
    else:
        return 'spam'

In [8]:
tp, tn, fp, fn = 0, 0, 0, 0
for ans, bow in test:
    pred = predict(bow, spam_words, ham_words, n_spam, n_ham, prior_spam, prior_ham, alpha)

    if pred == 'spam':
        if ans == 'spam':
            tp += 1
        else:
            fp += 1
    else:
        if ans =='ham':
            tn += 1
        else:
            fn += 1

In [10]:
print(tp, tn, fp, fn)

165 797 149 4


In [9]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = (tp) / (tp + fp)
recall = (tp) / (tp + fn)
f1 = 2 * precision * recall /(precision + recall)

print("accuracy: ", accuracy) 
print("precision: ", precision) 
print("recall: ", recall) 
print("f1: ", f1)

accuracy:  0.862780269058296
precision:  0.5254777070063694
recall:  0.9763313609467456
f1:  0.6832298136645963
